In [2]:
import pandas as pd
import numpy as np

# Query 1

In [69]:
data=pd.read_excel(r"G:\dbquery-optimizer\notebooks\Query_Plan_stats(operator-based).xls")
data=data.iloc[:,1:]
data

,Query_No,No Plan,Default Plan,Plan 1,Plan 2,Plan 3,Plan 4,mean_duration,min_duration,max_duration,std deviation
0,1,74.00,2625.00,78.00,78.00,85.75,31.75,495.416667,31.75,2625.00,1043.456188
1,3,93.50,105.50,848.00,1968.50,828.25,125.25,661.500000,93.50,1968.50,733.506765
2,4,125.00,152.75,144.75,144.50,144.50,35.25,124.458333,35.25,152.75,44.663021
3,6,62.50,70.75,86.00,78.00,74.25,35.00,67.750000,35.00,86.00,17.829750
4,9,97.75,113.00,164.00,171.75,164.00,117.50,138.000000,97.75,171.75,32.113471
5,14,82.50,97.75,94.00,97.50,97.75,15.50,80.833333,15.50,97.75,32.541384
6,19,62.25,62.00,74.00,70.00,78.50,23.25,61.666667,23.25,78.50,19.905820
7,20,58.75,78.00,82.25,78.00,82.25,113.50,82.125000,58.75,113.50,17.690923
8,22,78.00,93.75,93.50,93.75,94.00,31.00,80.666667,31.00,94.00,25.134472
9,23,86.00,93.50,93.50,97.75,94.00,31.00,82.625000,31.00,97.75,25.578189


## Data preparation

In [70]:
def extract_parameters(query):
    params=[]
    s=query
    s_index=query.find('cn.country_code = ')+len('cn.country_code = ')
    param1=''
    while(s[s_index]!='\n'):
        param1+=s[s_index]
        s_index+=1
    # param1=param1.replace("'","''")
    params.append(param1)
    s_index=query.find("rt.role = '")+len("rt.role = '")
    param2=''
    while(s[s_index]!="'"):
        param2+=s[s_index]
        s_index+=1
    params.append(param2)
    s_index=query.find('t.production_year > ')+len('t.production_year > ')
    param3=''
    while(s[s_index]!='\n'):
        param3+=s[s_index]
        s_index+=1
    params.append(param3)
    return params

In [71]:
n_data=[]
labels=[]
for i in range(data.shape[0]-1):
    f=open("G:\dbquery-optimizer\query-variants\query_1\q_one_"+str(data['Query_No'][i])+".sql",'r')
    qry=f.read()
    params=extract_parameters(qry)
    t=data['min_duration'][i]
    best_plan=""
    if(t==data['No Plan'][i]):best_plan+="No Plan"
    elif(t==data['Default Plan'][i]):best_plan+="Default Plan"
    elif(t==data['Plan 1'][i]):best_plan+="Plan 1"
    elif(t==data['Plan 2'][i]):best_plan+="Plan 2"
    elif(t==data['Plan 3'][i]):best_plan+="Plan 3"
    elif(t==data['Plan 4'][i]):best_plan+="Plan 4"
    labels.append(best_plan)
    n_data.append(params)

n_data=np.array(n_data)
labels=np.array(labels)
df=pd.DataFrame({'Parameter 1':n_data[:,0],'Parameter 2':n_data[:,1],'Parameter 3':n_data[:,2],'Best Plan':labels})
df['Parameter 3']=pd.to_numeric(df['Parameter 3'])
df

,Parameter 1,Parameter 2,Parameter 3,Best Plan
0,'[ph]',cinematographer,1923,Plan 4
1,'[ve]',editor,1974,No Plan
2,'[pt]',director,1999,Plan 4
3,'[kr]',director,1912,Plan 4
4,'[us]',producer,1988,No Plan
5,'[ve]',composer,2015,Plan 4
6,'[us]',producer,1942,Plan 4
7,'[id]',cinematographer,1906,No Plan
8,'[it]',producer,1980,Plan 4
9,'[kr]',guest,1930,Plan 4


In [72]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([dummies,original_dataframe], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features=['Parameter 1','Parameter 2']
for f in features:
    df=encode_and_bind(df,f)

df

,Parameter 2_actor,Parameter 2_cinematographer,Parameter 2_composer,Parameter 2_designer,Parameter 2_director,Parameter 2_editor,Parameter 2_guest,Parameter 2_producer,Parameter 2_writer,Parameter 1_'[id]',Parameter 1_'[it]',Parameter 1_'[kr]',Parameter 1_'[no]',Parameter 1_'[ph]',Parameter 1_'[pl]',Parameter 1_'[pt]',Parameter 1_'[us]',Parameter 1_'[ve]',Parameter 3,Best Plan
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1923,Plan 4
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1974,No Plan
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1999,Plan 4
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1912,Plan 4
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1988,No Plan
5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2015,Plan 4
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1942,Plan 4
7,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1906,No Plan
8,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1980,Plan 4
9,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1930,Plan 4


In [73]:
from imblearn.over_sampling import RandomOverSampler
oversample=RandomOverSampler(random_state=42)
X=df.iloc[:,:-1]
y=df['Best Plan']
X=np.array(X)
y=np.array(y)
X,y=oversample.fit_sample(X,y)
print(X.shape,y.shape)

(22, 19) (22,)


In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,criterion='gini')
clf.fit(X_train,y_train)

RandomForestClassifier()

In [94]:
y_pred=clf.predict(X_test)
y_pred

array(['No Plan', 'No Plan', 'No Plan', 'Plan 4', 'No Plan'], dtype=object)

In [95]:
y_test

array(['No Plan', 'Plan 4', 'No Plan', 'Plan 4', 'No Plan'], dtype=object)

In [96]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8


In [103]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [104]:
y_pred=clf.predict(X_test)
print(y_pred, y_test)
accuracy_score(y_test,y_pred)

['No Plan' 'No Plan' 'No Plan' 'Plan 4' 'No Plan'] ['No Plan' 'Plan 4' 'No Plan' 'Plan 4' 'No Plan']


0.8

# Query 2

In [45]:
data=pd.read_excel(r"G:\dbquery-optimizer\notebooks\Query_2_Plan_stats(operator-based).xls")
data=data.iloc[:,1:]
data

,Query_No,No Plan,Default Plan,Plan 1,Plan 2,Plan 3,Plan 4,Plan 5,mean_duration,min_duration,max_duration,std deviation
0,2,238.25,308.25,226.75,179.75,179.50,183.25,250.25,223.714286,179.50,308.25,47.602615
1,5,297.00,277.25,273.25,250.00,230.50,238.25,261.50,261.107143,230.50,297.00,23.377721
2,6,195.25,250.00,301.25,199.00,234.50,199.00,261.25,234.321429,195.25,301.25,39.732888
3,9,211.00,273.50,250.00,246.00,246.00,167.75,261.50,236.535714,167.75,273.50,35.901610
4,13,218.75,261.50,289.00,187.50,238.50,195.50,288.75,239.928571,187.50,289.00,41.693503
5,14,191.50,207.00,301.00,203.25,230.25,179.75,289.25,228.857143,179.75,301.00,47.948509
6,15,293.00,324.00,304.75,1484.50,308.50,1382.75,324.00,631.642857,293.00,1484.50,548.752332
7,18,245.75,324.00,265.75,160.00,257.50,195.00,273.50,245.928571,160.00,324.00,53.789078
8,22,195.50,300.75,258.00,226.50,261.50,191.50,258.00,241.678571,191.50,300.75,39.361232
9,23,211.00,230.25,242.25,171.75,238.00,195.50,250.00,219.821429,171.75,250.00,28.381961


In [46]:
def extract_parameters(query):
    params=[]
    s=query
    s_index=query.find("it.info ='")+len("it.info ='")
    param1=''
    while(s[s_index]!="'"):
        param1+=s[s_index]
        s_index+=1
    params.append(param1)
    s_index=query.find("mi_idx.info > '")+len("mi_idx.info > '")
    param2=''
    while(s[s_index]!="'"):
        param2+=s[s_index]
        s_index+=1
    params.append(param2)
    s_index=query.find('t.production_year > ')+len('t.production_year > ')
    param3=''
    while(s[s_index]!='\n'):
        param3+=s[s_index]
        s_index+=1
    params.append(param3)
    return params

In [47]:
n_data=[]
labels=[]
for i in range(data.shape[0]-1):
    f=open("G:\dbquery-optimizer\query-variants\query_2\q_two_"+str(data['Query_No'][i])+".sql",'r')
    qry=f.read()
    params=extract_parameters(qry)
    t=data['min_duration'][i]
    best_plan=""
    if(t==data['No Plan'][i]):best_plan+="No Plan"
    elif(t==data['Default Plan'][i]):best_plan+="Default Plan"
    elif(t==data['Plan 1'][i]):best_plan+="Plan 1"
    elif(t==data['Plan 2'][i]):best_plan+="Plan 2"
    elif(t==data['Plan 3'][i]):best_plan+="Plan 3"
    elif(t==data['Plan 4'][i]):best_plan+="Plan 4"
    elif(t==data['Plan 5'][i]):best_plan+="Plan 5"
    labels.append(best_plan)
    n_data.append(params)

n_data=np.array(n_data)
labels=np.array(labels)
df=pd.DataFrame({'Parameter 1':n_data[:,0],'Parameter 2':n_data[:,1],'Parameter 3':n_data[:,2],'Best Plan':labels})
df['Parameter 2']=pd.to_numeric(df['Parameter 2'])
df['Parameter 3']=pd.to_numeric(df['Parameter 3'])
df

,Parameter 1,Parameter 2,Parameter 3,Best Plan
0,novel,182.6,1907,Plan 3
1,locations,350.8,1989,Plan 3
2,rating,315.9,1948,No Plan
3,rating,399.5,1961,Plan 4
4,rentals,303.5,1999,Plan 2
5,plot,195.0,1938,Plan 4
6,books,973.0,1991,No Plan
7,novel,146.2,1944,Plan 2
8,locations,187.0,1960,Plan 4
9,locations,858.7,1932,Plan 2


In [48]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([dummies,original_dataframe], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features=['Parameter 1']
for f in features:
    df=encode_and_bind(df,f)

df

,Parameter 1_article,Parameter 1_books,Parameter 1_locations,Parameter 1_novel,Parameter 1_plot,Parameter 1_rating,Parameter 1_rentals,Parameter 1_trivia,Parameter 2,Parameter 3,Best Plan
0,0,0,0,1,0,0,0,0,182.6,1907,Plan 3
1,0,0,1,0,0,0,0,0,350.8,1989,Plan 3
2,0,0,0,0,0,1,0,0,315.9,1948,No Plan
3,0,0,0,0,0,1,0,0,399.5,1961,Plan 4
4,0,0,0,0,0,0,1,0,303.5,1999,Plan 2
5,0,0,0,0,1,0,0,0,195.0,1938,Plan 4
6,0,1,0,0,0,0,0,0,973.0,1991,No Plan
7,0,0,0,1,0,0,0,0,146.2,1944,Plan 2
8,0,0,1,0,0,0,0,0,187.0,1960,Plan 4
9,0,0,1,0,0,0,0,0,858.7,1932,Plan 2


In [50]:
from imblearn.over_sampling import RandomOverSampler
oversample=RandomOverSampler(random_state=42)
X=df.iloc[:,:-1]
y=df['Best Plan']
X=np.array(X)
y=np.array(y)
X,y=oversample.fit_sample(X,y)
print(X.shape,y.shape)

(56, 10) (56,)


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,criterion='gini')
clf.fit(X_train,y_train)

RandomForestClassifier()

In [63]:
y_pred=clf.predict(X_test)
print(y_pred)
print(y_test)

['Plan 5' 'No Plan' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 5'
 'No Plan' 'Plan 5' 'Plan 1' 'Plan 3']
['Plan 5' 'No Plan' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 5'
 'Plan 2' 'No Plan' 'Plan 1' 'Plan 4']


In [64]:
print(accuracy_score(y_test,y_pred))

0.75


In [65]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(y_pred,y_test)
print(accuracy_score(y_test,y_pred))

['Plan 5' 'No Plan' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 5'
 'Plan 2' 'Plan 5' 'Plan 1' 'Plan 4'] ['Plan 5' 'No Plan' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 1' 'Plan 3' 'Plan 5'
 'Plan 2' 'No Plan' 'Plan 1' 'Plan 4']
0.9166666666666666


# Query 3

In [105]:
data=pd.read_excel(r"G:\dbquery-optimizer\notebooks\Query_3_Plan_stats(operator-based).xls")
data=data.iloc[:,1:]
data

,Query_No,No Plan,Default Plan,Plan 1,Plan 2,Plan 3,mean_duration,min_duration,max_duration,std deviation
0,1,19.25,39.00,23.50,609.25,35.00,145.20,19.25,609.25,259.537654
1,3,191.25,382.75,47.00,2793.00,62.75,695.35,47.00,2793.00,1180.308389
2,5,15.75,35.00,27.25,492.25,35.00,121.05,15.75,492.25,207.656673
3,7,15.75,35.00,31.25,2500.00,39.00,524.20,15.75,2500.00,1104.540903
4,8,15.75,31.25,27.50,453.00,31.25,111.75,15.75,453.00,190.870457
5,9,55.25,23.00,54.75,1890.50,23.75,409.45,23.00,1890.50,828.083167
6,15,4.75,23.75,27.75,441.75,23.00,104.20,4.75,441.75,188.905102
7,19,27.25,42.75,46.75,2284.75,51.00,490.50,27.25,2284.75,1003.056329
8,22,8.25,31.25,31.25,480.25,43.00,118.80,8.25,480.25,202.449407
9,24,4.75,23.75,27.25,456.75,35.25,109.55,4.75,456.75,194.413413


In [106]:
def extract_parameters(query):
    params=[]
    s=query
    s_index=query.find("cn.country_code ='")+len("cn.country_code ='")
    param1=''
    while(s[s_index]!="'"):
        param1+=s[s_index]
        s_index+=1
    params.append(param1)
    s_index=query.find("ct.kind ='")+len("ct.kind ='")
    param2=''
    while(s[s_index]!="'"):
        param2+=s[s_index]
        s_index+=1
    params.append(param2)
    s_index=query.find("it.info ='")+len("it.info ='")
    param3=''
    while(s[s_index]!="'"):
        param3+=s[s_index]
        s_index+=1
    params.append(param3)
    s_index=query.find("kt.kind ='")+len("kt.kind ='")
    param4=''
    while(s[s_index]!="'"):
        param4+=s[s_index]
        s_index+=1
    params.append(param4)
    return params

In [107]:
n_data=[]
labels=[]
for i in range(data.shape[0]-1):
    f=open("G:\dbquery-optimizer\query-variants\query_3\q_three_"+str(data['Query_No'][i])+".sql",'r')
    qry=f.read()
    params=extract_parameters(qry)
    t=data['min_duration'][i]
    best_plan=""
    if(t==data['No Plan'][i]):best_plan+="No Plan"
    elif(t==data['Default Plan'][i]):best_plan+="Default Plan"
    elif(t==data['Plan 1'][i]):best_plan+="Plan 1"
    elif(t==data['Plan 2'][i]):best_plan+="Plan 2"
    elif(t==data['Plan 3'][i]):best_plan+="Plan 3"
    labels.append(best_plan)
    n_data.append(params)

n_data=np.array(n_data)
labels=np.array(labels)
df=pd.DataFrame({'Parameter 1':n_data[:,0],'Parameter 2':n_data[:,1],'Parameter 3':n_data[:,2],'Parameter 4':n_data[:,3],'Best Plan':labels})
df

,Parameter 1,Parameter 2,Parameter 3,Parameter 4,Best Plan
0,[az],distributors,trivia,episode,No Plan
1,[us],special effects companies,plot,tv mini series,Plan 1
2,[us],miscellaneous companies,novel,tv series,No Plan
3,[id],production companies,books,tv mini series,No Plan
4,[kr],production companies,locations,movie,No Plan
5,[as],production companies,plot,tv series,Default Plan
6,[ug],distributors,locations,tv series,No Plan
7,[in],production companies,rating,episode,No Plan
8,[ps],distributors,rating,tv series,No Plan
9,[ps],distributors,books,tv movie,No Plan


In [108]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([dummies,original_dataframe], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features=['Parameter 1','Parameter 2','Parameter 3','Parameter 4']
for f in features:
    df=encode_and_bind(df,f)

df

,Parameter 4_episode,Parameter 4_movie,Parameter 4_tv mini series,Parameter 4_tv movie,Parameter 4_tv series,Parameter 4_video game,Parameter 4_video movie,Parameter 3_article,Parameter 3_books,Parameter 3_locations,...,Parameter 1_[id],Parameter 1_[in],Parameter 1_[kr],Parameter 1_[pl],Parameter 1_[ps],Parameter 1_[pt],Parameter 1_[tl],Parameter 1_[ug],Parameter 1_[us],Best Plan
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,No Plan
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Plan 1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,No Plan
3,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,No Plan
4,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,No Plan
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Default Plan
6,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,No Plan
7,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,No Plan
8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,No Plan
9,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,No Plan


In [110]:
from imblearn.over_sampling import RandomOverSampler
oversample=RandomOverSampler(random_state=42)
X=df.iloc[:,:-1]
y=df['Best Plan']
X=np.array(X)
y=np.array(y)
X,y=oversample.fit_sample(X,y)
print(X.shape,y.shape)

(57, 32) (57,)


In [111]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,criterion='gini')
clf.fit(X_train,y_train)

RandomForestClassifier()

In [112]:
y_pred=clf.predict(X_test)
print(y_pred,y_test)

['Default Plan' 'Plan 1' 'Plan 1' 'Default Plan' 'Plan 1' 'Plan 1'
 'Plan 1' 'No Plan' 'Plan 1' 'No Plan' 'Plan 1' 'Default Plan'] ['Default Plan' 'Plan 1' 'Plan 1' 'No Plan' 'Plan 1' 'Plan 1' 'Plan 1'
 'No Plan' 'Plan 1' 'No Plan' 'Plan 1' 'Default Plan']


In [113]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9166666666666666


In [114]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(y_pred,y_test)
accuracy_score(y_test,y_pred)

['Default Plan' 'Plan 1' 'Plan 1' 'Default Plan' 'Plan 1' 'Plan 1'
 'Plan 1' 'No Plan' 'Plan 1' 'No Plan' 'Plan 1' 'Default Plan'] ['Default Plan' 'Plan 1' 'Plan 1' 'No Plan' 'Plan 1' 'Plan 1' 'Plan 1'
 'No Plan' 'Plan 1' 'No Plan' 'Plan 1' 'Default Plan']


0.9166666666666666

# Query 5

In [1]:
data=pd.read_excel(r"G:\dbquery-optimizer\notebooks\Query_5_Plan_stats(operator-based).xls")
data=data.iloc[:,1:]
data

NameError: name 'pd' is not defined

In [118]:
def extract_parameters(query):
    params=[]
    s=query
    s_index=query.find("cn.country_code ='")+len("cn.country_code ='")
    param1=''
    while(s[s_index]!="'"):
        param1+=s[s_index]
        s_index+=1
    params.append(param1)
    s_index=query.find("k.keyword ='")+len("k.keyword ='")
    param2=''
    while(s[s_index]!="'"):
        param2+=s[s_index]
        s_index+=1
    params.append(param2)
    return params


In [117]:
n_data=[]
labels=[]
for i in range(data.shape[0]-1):
    f=open("G:\dbquery-optimizer\query-variants\query_5\q_five_"+str(data['Query_No'][i])+".sql",'r')
    qry=f.read()
    params=extract_parameters(qry)
    t=data['min_duration'][i]
    best_plan=""
    if(t==data['No Plan'][i]):best_plan+="No Plan"
    elif(t==data['Default Plan'][i]):best_plan+="Default Plan"
    elif(t==data['Plan 1'][i]):best_plan+="Plan 1"
    elif(t==data['Plan 2'][i]):best_plan+="Plan 2"
    elif(t==data['Plan 3'][i]):best_plan+="Plan 3"
    labels.append(best_plan)
    n_data.append(params)

n_data=np.array(n_data)
labels=np.array(labels)
df=pd.DataFrame({'Parameter 1':n_data[:,0],'Parameter 2':n_data[:,1],'Best Plan':labels})
df

,Parameter 1,Parameter 2,Best Plan
0,[us],character-name-in-title,Plan 3
1,[no],character-name-in-title,Plan 2
2,[in],black-jew,Default Plan
3,[tl],public-hearing,Plan 3
4,[cr],rabbit-fight,Plan 3
5,[kr],black-jew,Default Plan
6,[us],character-name-in-title,Plan 2
7,[ve],character-name-in-title,Plan 2
8,[no],home-alone,Plan 3
9,[om],character-name-in-title,No Plan


In [31]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([dummies,original_dataframe], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features=['Parameter 1','Parameter 2']
for f in features:
    df=encode_and_bind(df,f)

df

,Parameter 2_black-jew,Parameter 2_character-name-in-title,Parameter 2_fictional-plot,Parameter 2_home-alone,Parameter 2_middle-ages,Parameter 2_public-hearing,Parameter 2_rabbit-fight,Parameter 1_[by],Parameter 1_[cr],Parameter 1_[id],...,Parameter 1_[lk],Parameter 1_[no],Parameter 1_[om],Parameter 1_[ph],Parameter 1_[ps],Parameter 1_[pt],Parameter 1_[tl],Parameter 1_[us],Parameter 1_[ve],Best Plan
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Plan 3
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Plan 2
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Default Plan
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Plan 3
4,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,Plan 3
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Default Plan
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Plan 2
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Plan 2
8,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Plan 3
9,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,No Plan


In [32]:
from imblearn.over_sampling import RandomOverSampler
oversample=RandomOverSampler(random_state=42)
X=df.iloc[:,:-1]
y=df['Best Plan']
X=np.array(X)
y=np.array(y)
X,y=oversample.fit_sample(X,y)
print(X.shape,y.shape)

(44, 22) (44,)


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100,criterion='gini')
clf.fit(X_train,y_train)

RandomForestClassifier()

In [43]:
y_pred=clf.predict(X_test)
print(y_pred,y_test)

['Plan 3' 'Default Plan' 'No Plan' 'Plan 2' 'Plan 2' 'Plan 2'
 'Default Plan' 'Plan 2' 'Default Plan'] ['Plan 3' 'Default Plan' 'Plan 3' 'Plan 2' 'Plan 2' 'Plan 3'
 'Default Plan' 'Plan 2' 'Default Plan']


In [44]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7777777777777778


In [28]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [29]:
y_pred=clf.predict(X_test)
print(y_pred,y_test)
print(accuracy_score(y_test,y_pred))

['Default Plan' 'Plan 2' 'Default Plan' 'Plan 3' 'Plan 3' 'Plan 2'] ['Plan 2' 'Plan 2' 'Plan 3' 'Plan 3' 'No Plan' 'Plan 3']
0.3333333333333333
